In [8]:
import optlang
import numpy as np
import sympy

from optlang import Model, Variable, Constraint, Objective

import sys
sys.path.append("..")
from neurop.Problem.ILPProblem import ILPProblem

# All the (symbolic) variables are declared, with a name and optionally a lower and/or upper bound.
x1 = Variable('x1', lb=5, ub=100, type="integer")
x2 = Variable('x2', lb=0, ub=100, type="integer")
x3 = Variable('x3', lb=0, ub=100, type="integer")

# A constraint is constructed from an expression of variables and a lower and/or upper bound (lb and ub).
c1 = Constraint(x1 + x2 + x3, ub=100)
c2 = Constraint(10 * x1 + 4 * x2 + 5 * x3, ub=600)
c3 = Constraint(2 * x1 + 2 * x2 + 6 * x3, ub=300)

# An objective can be formulated
obj = Objective(10 * x1 + 6 * x2 + 4 * x3, direction='max')

# Variables, constraints and objective are combined in a Model object, which can subsequently be optimized.
model = Model(name='Simple model')
model.objective = obj
model.add([c1, c2, c3])

status = model.optimize()

opt_sol = {}
print("status:", model.status)
print("objective value:", model.objective.value)
print("----------")
for var_name, var in model.variables.iteritems():
    print(var_name, "=", var.primal)
    opt_sol[var] = var.primal

status: optimal
objective value: 732.0
----------
x1 = 33.0
x2 = 67.0
x3 = 0.0


In [9]:
problem = ILPProblem(model)
Q = problem.to_qubo()
Q

Matrix([
[105770.0,    2120.0,    4240.0,    8480.0,   16960.0,    33920.0,    67840.0,    450.0,    900.0,   1800.0,    3600.0,    7200.0,   14400.0,    28800.0,    630.0,   1260.0,    2520.0,    5040.0,   10080.0,   20160.0,    40320.0,   -20.0,   -40.0,    -80.0,   -160.0,   -320.0,    -640.0,   -1280.0,   -2560.0,   -5120.0,  -100.0,   -200.0,   -400.0,   -800.0,   -1600.0,   -3200.0,   -6400.0,   -12800.0,   -25600.0,   -51200.0,  -10.0,   -20.0,   -40.0,   -80.0,   -160.0,   -320.0,   -640.0,    0,     0,     0,     0,      0,      0,      0,    0,     0,     0,     0,      0,      0,      0,    -10,     -20,     -40,     -80,    -160,     -320,   -640],
[  2120.0,  213660.0,    8480.0,   16960.0,   33920.0,    67840.0,   135680.0,    900.0,   1800.0,   3600.0,    7200.0,   14400.0,   28800.0,    57600.0,   1260.0,   2520.0,    5040.0,   10080.0,   20160.0,   40320.0,    80640.0,   -40.0,   -80.0,   -160.0,   -320.0,   -640.0,   -1280.0,   -2560.0,   -5120.0,  -10240.0,  -200.0, 

In [10]:
var_a = sympy.Array(problem.variables, (len(problem.variables),1))
(var_a.transpose() @ Q @ var_a)[0].expand()

-4410.0*s_0_0**2 + 40*s_0_0*s_0_1 + 80*s_0_0*s_0_2 + 160*s_0_0*s_0_3 + 320*s_0_0*s_0_4 + 640*s_0_0*s_0_5 + 1280*s_0_0*s_0_6 + 2560*s_0_0*s_0_7 + 5120*s_0_0*s_0_8 - 40.0*s_0_0*x1_0 - 80.0*s_0_0*x1_1 - 160.0*s_0_0*x1_2 - 320.0*s_0_0*x1_3 - 640.0*s_0_0*x1_4 - 1280.0*s_0_0*x1_5 - 2560.0*s_0_0*x1_6 - 40.0*s_0_0*x2_0 - 80.0*s_0_0*x2_1 - 160.0*s_0_0*x2_2 - 320.0*s_0_0*x2_3 - 640.0*s_0_0*x2_4 - 1280.0*s_0_0*x2_5 - 2560.0*s_0_0*x2_6 - 120.0*s_0_0*x3_0 - 240.0*s_0_0*x3_1 - 480.0*s_0_0*x3_2 - 960.0*s_0_0*x3_3 - 1920.0*s_0_0*x3_4 - 3840.0*s_0_0*x3_5 - 7680.0*s_0_0*x3_6 - 8800.0*s_0_1**2 + 160*s_0_1*s_0_2 + 320*s_0_1*s_0_3 + 640*s_0_1*s_0_4 + 1280*s_0_1*s_0_5 + 2560*s_0_1*s_0_6 + 5120*s_0_1*s_0_7 + 10240*s_0_1*s_0_8 - 80.0*s_0_1*x1_0 - 160.0*s_0_1*x1_1 - 320.0*s_0_1*x1_2 - 640.0*s_0_1*x1_3 - 1280.0*s_0_1*x1_4 - 2560.0*s_0_1*x1_5 - 5120.0*s_0_1*x1_6 - 80.0*s_0_1*x2_0 - 160.0*s_0_1*x2_1 - 320.0*s_0_1*x2_2 - 640.0*s_0_1*x2_3 - 1280.0*s_0_1*x2_4 - 2560.0*s_0_1*x2_5 - 5120.0*s_0_1*x2_6 - 240.0*s_0_1*x3_

In [11]:
import neal
import dimod

bqm=dimod.BinaryQuadraticModel(np.array(Q,dtype=int), "BINARY")
sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, beta_schedule_type="geometric", num_reads=1000, num_sweeps=1000)
sampleset.relabel_variables(dict(enumerate(problem.variables)))
#decoded_samples = model.decode_sampleset(sampleset)
#best_sample = min(decoded_samples, key=lambda x: x.energy)
#pprint(best_sample.sample)
best=sampleset.lowest().first.sample

sol_bin = np.array([best[v] for v in problem.variables])
sol_bin

array([0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 0], dtype=int8)

In [12]:
problem.evaluate_objective(sol_bin)

-678.000000000000

In [13]:
sol_int=problem.parameters_from_qubo(sol_bin)
#substitute_binary_to_integer(sympy.Matrix(sol_bin), substitutions)
sol_int

{5 <= x1 <= 100: 29, 0 <= x2 <= 100: 62, 0 <= x3 <= 100: 4}

In [14]:
print("==== Results ====")
for k,v in sol_int.items():
    print("{}: {} (optimal: {})".format(k.name, v, opt_sol[k]))

==== Results ====
x1: 29 (optimal: 33.0)
x2: 62 (optimal: 67.0)
x3: 4 (optimal: 0.0)
